## Cross Validation

This notebook looks at polynomial regression in addition to simple linear regression, and uses 10 K-folds to cross validate a test set after being fit on a training set.

In [4]:
import numpy as np
import pandas as pd

import statsmodels.formula.api as smf

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

from sklearn.model_selection import KFold

In [3]:
df = pd.read_csv('./data/transcoz6_winter.csv', usecols=["Log_Price","Low_Temp_log"])
df.head(10)

,Log_Price,Low_Temp_log
0,0.438255,3.760037
1,0.329304,3.872034
2,0.307485,3.997466
3,0.277632,4.168679
4,0.451076,4.090337
5,0.570980,3.938275
6,0.494696,3.918601
7,0.425268,3.961575
8,0.582216,3.988984
9,0.615186,3.775745


In [51]:
X, y = df['Log_Price'], df['Low_Temp_log']

#hold out 20% of the data for final testing
X, X_test, y, y_test = train_test_split(X, y, test_size=.2, random_state=10)

#this helps with the way kf will generate indices below
X, y = np.array(X), np.array(y)
X_test, y_test = np.array(X_test), np.array(y_test)

In [69]:
kf = KFold(n_splits=10, shuffle=True, random_state = 33) 
#shuffle True will randomize the data before doing the splits

cv_lm_r2s, cv_lm_poly_r2s = [], [] #collect the validation results for both models

for train_ind, val_ind in kf.split(X,y):
    
    X_train, y_train = X[train_ind], y[train_ind]
    X_val, y_val = X[val_ind], y[val_ind] 
    
    #simple linear regression
    lm = LinearRegression()

    lm.fit(X_train.reshape(-1, 1), y_train)
    cv_lm_r2s.append(lm.score(X_val.reshape(-1, 1), y_val))
    
    #need to reshape everything because there's only one feature

    #polynomial regression with degree 2
    poly = PolynomialFeatures(degree=2) 

    X_train_poly = poly.fit_transform(X_train.reshape(-1, 1))
    X_val_poly = poly.transform(X_val.reshape(-1, 1))
    X_test_poly = poly.transform(X_test.reshape(-1, 1))

    lm_poly = LinearRegression()
    lm_poly.fit(X_train_poly, y_train)
    cv_lm_poly_r2s.append(lm_poly.score(X_val_poly, y_val))

print('Simple regression scores: ', cv_lm_r2s)
print('Degree 2 polynomial scores: ', cv_lm_poly_r2s, '\n')

print(f'Simple mean cv r^2: {np.mean(cv_lm_r2s):.3f} +- {np.std(cv_lm_r2s):.3f}')
print(f'Poly mean cv r^2: {np.mean(cv_lm_poly_r2s):.3f} +- {np.std(cv_lm_poly_r2s):.3f}')

Simple regression scores:  [0.5183587548879174, 0.4506757535733857, 0.1658512098388497, 0.5587721680370903, 0.3696824137219281, 0.5895953788783432, 0.5146110749738777, 0.6951350331026895, 0.04577950889948834, 0.11514092978113788]
Degree 2 polynomial scores:  [0.5269701255483132, 0.4564372416541913, 0.17096071331633267, 0.5554007474238754, 0.3382171358923485, 0.5783170225913746, 0.46681266572211577, 0.6971200982980928, 0.020637748676765155, 0.1233101170564971] 

Simple mean cv r^2: 0.402 +- 0.210
Poly mean cv r^2: 0.393 +- 0.211


In [72]:
lm.fit(X.reshape(-1,1),y)
print(f'Linear Regression test R^2: {lm.score(X_test.reshape(-1,1), y_test):.3f}')

Linear Regression test R^2: 0.536


## Conclusion

Overall, the simple linear regression performed just as well as the polynomial regression, but is simpler and less computationally expensive.  The test set even had a higher R-squared than the training set!  
Compared to other natural gas hubs, **Transco Z6 NY** spot prices in the winter are pretty correlated to the daily low temperatures by its nearest physical location in **Linden, NJ**, and can be roughly predicted within a certain range.  
I believe this is due to a variety of reasons:
- the largest demand location (New York City) is located near by
- winters are cold enough to warrant a larger increase in demand (as opposed to heating demand in Louisiana)
- there aren't many other options to supply gas outside of Transco Z6